# Evaluating models based on medQA dataset

Using the https://huggingface.co/datasets/medmcqa/viewer/default/validation?row=0 dataset we asses the models performance on the medQA dataset.

## Libraries & Setup

### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

from datasets import load_dataset

/opt/homebrew/anaconda3/envs/adl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from chat_doc.inference.deploy_endpoint import SageMakerDeployment
from chat_doc.inference.chat import Chat

### Model loading

In [ ]:
s3_model_uri = "s3://sagemaker-eu-central-1-228610994900/huggingface-qlora-2023-12-08-15-01-13-2023-12-08-15-01-14-300/output/model.tar.gz"
deployment = SageMakerDeployment(s3_model_uri)
llm_endpoint = deployment.deploy_model()

### Setup

In [ ]:
chat = Chat(llm_endpoint)

### Data Loading

In [2]:
dataset = load_dataset("medmcqa")
validation = dataset["validation"]

Generating validation split: 100%|██████████| 4183/4183 [00:00<00:00, 35396.36 examples/s]


## Evaluation

In [3]:
validation

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 4183
})

In [4]:
def build_qa_query(row):
    return f"""Please answer the {row['choice_type']}-choice question to the best of your knowledge by just returning the correct option. The subject is {row['subject']}.

    Question: {row['question']}
    Options: 
    A) {row['opa']}
    B) {row['opb']}
    C) {row['opc']}
    D) {row['opd']}"""

In [ ]:
chat.predict(
    build_qa_query(validation[0])
)